In [1]:
import pandas as pd 
import numpy as np 
import gpflow
import tensorflow as tf
from GPcounts import NegativeBinomialLikelihood
from GPcounts.GPcounts_Module import Fit_GPcounts
from matplotlib import pyplot as plt
import scanpy as sc
import seaborn as sns
import numpy as np
from matplotlib import pyplot as plt
from scipy.cluster.vq import kmeans

In [2]:
adata = sc.datasets.visium_sge(
    sample_id="V1_Mouse_Brain_Sagittal_Anterior")

/Users/user/opt/anaconda3/envs/tf/lib/python3.8/site-packages/anndata/_core/anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [3]:
adata.var_names_make_unique()
adata.var['mt'] = adata.var_names.str.startswith('MT-')
sc.pp.calculate_qc_metrics(adata,qc_vars = ["mt"], inplace=True)

In [4]:
# adata.obsm['spatial']
spatialx = [str(i) for i in adata.obsm['spatial'][:,0]]
spatialy = [str(i) for i in adata.obsm['spatial'][:,1]]

In [5]:
index_names = [i+'x'+j for i,j in zip(spatialx,spatialy)]

In [6]:
mito_genes = adata.var_names.str.startswith('mt-')
keep = np.invert(mito_genes)
adata = adata[:,keep]
print(adata.n_obs, adata.n_vars)

2695 32272


In [7]:
Y = pd.DataFrame(data=adata.X.A, index=index_names, columns = adata.var.index)

In [8]:
Y = Y.T[Y.sum(0) >= 10].T # Filter out genes with less than 3 total counts

In [9]:
spatial_locations = pd.DataFrame(index=Y.index)
spatial_locations['x'] = Y.index.str.split('x').str.get(0).map(float)
spatial_locations['y'] = Y.index.str.split('x').str.get(1).map(float)

In [10]:
spatial_locations['total_counts'] = Y.sum(1)
Y = Y.loc[spatial_locations.index]
X = spatial_locations[['x', 'y']]

In [11]:
# import scipy as sp
# sp.special.wrightomega(np.log(X.shape[0]))

In [12]:
import statsmodels.formula.api as smf 
import statsmodels.api as sm
scales = []
for i in range(0,len(Y.columns)):
    model=smf.glm(formula = "Y.iloc[:,i]~0+spatial_locations['total_counts']", data=Y,family=sm.families.NegativeBinomial(sm.families.links.log())).fit()
    res = model.params[0]*spatial_locations['total_counts']
    scales.append(res)
scalesdf=pd.DataFrame(scales)
scalesdf=scalesdf.T
print(scalesdf)

           total_counts  total_counts  total_counts  total_counts  \
8500x7474     -2.126576     -2.020778     -0.748637      0.199988   
2788x8552     -6.106828     -5.803010     -2.149839      0.574299   
7950x3163     -4.380051     -4.162141     -1.541947      0.411909   
2100x6636     -6.729553     -6.394753     -2.369062      0.632861   
2375x7115     -5.775425     -5.488094     -2.033173      0.543133   
...                 ...           ...           ...           ...   
5541x7594     -4.467038     -4.244800     -1.572570      0.420089   
5885x4361     -4.517354     -4.292613     -1.590283      0.424821   
6780x5199     -3.622241     -3.442032     -1.275169      0.340643   
4371x8433     -2.294069     -2.179938     -0.807601      0.215739   
3339x6876     -2.359565     -2.242175     -0.830658      0.221898   

           total_counts  total_counts  total_counts  total_counts  \
8500x7474     -0.086521      0.308813      0.087101      0.613868   
2788x8552     -0.248461      0.88

In [13]:
scale = scalesdf

In [14]:
Y = Y.T
Y_run = Y.iloc[0:20,:] # select first 20 genes to run GPcounts
X = X[['x', 'y']]

In [16]:
gene_name = []
sparse = True 
nb_scaled = True # set the nb_scaled argument to True to pass the scale factors 
gene_name = Y_run.index
likelihood = 'Negative_binomial' 
gp_counts = Fit_GPcounts(X,Y_run.loc[gene_name], sparse = sparse, M=120,scale=scale,safe_mode=False)
# gp_counts.ConditionalVariance_inducing_points( M =5)

In [17]:
results = gp_counts.One_sample_test(likelihood)
results

 60%|█████████████████████████████████████████████████████████████████████████████████▌                                                      | 12/20 [02:24<01:47, 13.49s/it]2023-02-21 11:10:14.831039: W tensorflow/core/kernels/linalg/cholesky_op.cc:56] Cholesky decomposition was not successful. Eigen::LLT failed with error code 1. Filling lower-triangular output with NaNs.
2023-02-21 11:10:14.853844: W tensorflow/core/kernels/linalg/cholesky_op.cc:56] Cholesky decomposition was not successful. Eigen::LLT failed with error code 1. Filling lower-triangular output with NaNs.
2023-02-21 11:10:14.879575: W tensorflow/core/kernels/linalg/cholesky_op.cc:56] Cholesky decomposition was not successful. Eigen::LLT failed with error code 1. Filling lower-triangular output with NaNs.
2023-02-21 11:10:14.905647: W tensorflow/core/kernels/linalg/cholesky_op.cc:56] Cholesky decomposition was not successful. Eigen::LLT failed with error code 1. Filling lower-triangular output with NaNs.
2023-02-21 11:1

,Dynamic_model_log_likelihood,Constant_model_log_likelihood,log_likelihood_ratio
Xkr4,-322.509494,-319.117095,-3.392398
Gm19938,-416.036520,-354.339128,-61.697392
Sox17,-1409.040207,-1415.423273,6.383066
Mrpl15,-3962.921499,-4135.945898,173.024399
Lypla1,-2950.137930,-3033.112479,82.974549
Tcea1,-4415.196063,-4627.982357,212.786293
Rgs20,-3519.433489,-3805.459834,286.026345
Atp6v1h,-5637.303742,-6123.413475,486.109734
Oprk1,-1356.734147,-1640.386807,283.652660
Npbwr1,-258.766793,-260.180813,1.414020


In [18]:
results = gp_counts.calculate_FDR(results)

In [19]:
results

,Dynamic_model_log_likelihood,Constant_model_log_likelihood,log_likelihood_ratio,p_value,q_value
Xkr4,-322.509494,-319.117095,-3.392398,1.000000e+00,1.000000e+00
Gm19938,-416.036520,-354.339128,-61.697392,1.000000e+00,1.000000e+00
Sox17,-1409.040207,-1415.423273,6.383066,3.529518e-04,5.042169e-04
Mrpl15,-3962.921499,-4135.945898,173.024399,0.000000e+00,0.000000e+00
Lypla1,-2950.137930,-3033.112479,82.974549,0.000000e+00,0.000000e+00
Tcea1,-4415.196063,-4627.982357,212.786293,0.000000e+00,0.000000e+00
Rgs20,-3519.433489,-3805.459834,286.026345,0.000000e+00,0.000000e+00
Atp6v1h,-5637.303742,-6123.413475,486.109734,0.000000e+00,0.000000e+00
Oprk1,-1356.734147,-1640.386807,283.652660,0.000000e+00,0.000000e+00
Npbwr1,-258.766793,-260.180813,1.414020,9.263140e-02,1.157893e-01
